# check environment

In [1]:
import subprocess

# Command to list conda environments
conda_env_command = ["conda", "env", "list"]

# Command to print current directory
pwd_command = ["pwd"]

# Execute the commands
try:
    # Run the command to list conda environments
    subprocess.run(conda_env_command, check=True)

    # Run the command to print current directory
    subprocess.run(pwd_command, check=True)
except subprocess.CalledProcessError as e:
    print("An error occurred while executing the command:", e)


# conda environments:
#
base                     /opt/conda
jupyterlab               /opt/conda/envs/jupyterlab
pytorch                  /opt/conda/envs/pytorch
tensorflow               /opt/conda/envs/tensorflow
tvnEnv0005_cobra      *  /opt/conda/envs/tvnEnv0005_cobra

/home/jupyter/google_cloud/fba/formatotroph


# import data from google sheet

In [1]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import pandas as pd

# Configuration variables
SPREADSHEET_url = 'https://docs.google.com/spreadsheets/d/1LuksiakZgUk0-WDZ-GSwJFKvNZS7_Kv425EI-KMUfuI/edit?usp=sharing'
SERVICE_ACCOUNT_FILE = '/home/jupyter/google_cloud/fba/google_sheet/thuanguyen2-230332373cd1.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Extract the SPREADSHEET_ID from the URL
SPREADSHEET_ID = SPREADSHEET_url.split('/')[5]

def get_sheet_names(service, spreadsheet_id):
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = sheet_metadata.get('sheets', '')
    return [sheet.get("properties", {}).get("title", "") for sheet in sheets]

def read_sheet_to_dataframe(service, spreadsheet_id, sheet_name):
    range_name = f'{sheet_name}'
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])
    if values and len(values) > 1:
        return pd.DataFrame(values[1:], columns=values[0])
    else:
        print(f'No data found in sheet: {sheet_name}')
        return pd.DataFrame()

def main():
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet_names = get_sheet_names(service, SPREADSHEET_ID)
    for sheet_name in sheet_names:
        df = read_sheet_to_dataframe(service, SPREADSHEET_ID, sheet_name)
        # Dynamically assign DataFrame to a variable named after the sheet
        globals()[sheet_name.replace(" ", "_")] = df
        print(f'Sheet "{sheet_name}" has been read into a DataFrame and assigned to a variable with a similar name.')

# Execute the script
if __name__ == '__main__':
    main()


Sheet "stoimax" has been read into a DataFrame and assigned to a variable with a similar name.
Sheet "metabolite" has been read into a DataFrame and assigned to a variable with a similar name.
Sheet "reaction" has been read into a DataFrame and assigned to a variable with a similar name.


# import metabolic model

In [6]:
import cobra

# The ID of the iJO1366 model in the BiGG database
model_id = "iJO1366"

try:
    # Attempt to load the model from the BiGG database
    model = cobra.io.load_model(model_id, cache=True)
    print(f"Model {model_id} successfully loaded.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

# You can now work with the `model` object for your analyses


Model iJO1366 successfully loaded.


In [7]:
model

Name,iJO1366
Memory address,7f559a636510
Number of metabolites,1805
Number of reactions,2583
Number of genes,1367
Number of groups,37
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


# check metabolites in model

In [8]:
import pandas as pd
import cobra

# Get the list of metabolite IDs from the metabolite matrix
metabolite_ids_matrix = metabolite['id'].tolist()

# Get the list of metabolite IDs from the model
metabolite_ids_model = [met.id for met in model.metabolites]

# Find metabolites in the matrix that are not present in the model
metabolites_not_in_model = [met for met in metabolite_ids_matrix if met not in metabolite_ids_model]

# Print the list of metabolites not present in the model
print("Metabolites not present in the model:", metabolites_not_in_model)


Metabolites not present in the model: []


In [2]:
metabolite

,id,name
0,for_c,formate
1,thf_c,"5,6,7,8-Tetrahydrofolate"
2,10fthf_c,10-Formyltetrahydrofolate
3,methf_c,"5,10-Methenyltetrahydrofolate"
4,mlthf_c,"5,10-Methylenetetrahydrofolate"
5,gly_c,Glycine
6,ser__L_c,Serine
7,pyr_c,Pyruvate
8,nadh_c,Nicotinamide adenine dinucleotide
9,nad_c,Nicotinamide adenine dinucleotide - reduced
